In [1]:
'''
Overview
this notebook is a working example of scraping ufc fight stats one at a time
the code is broken down into small steps that can be run in parts to view and verify results at each step
this is useful for testing and debugging the code as the ufc stats website may be update, breaking the code


scrape ufc fight stats
get all event details, name, url, date, location for all ufc events
for each event, get fight details all fights on card
parse each fight to get fight stats of both fighters

additional scraping of fighter's details, name, nickname, url
for each fighter, get their tale of the tape
'''

In [54]:
# imports
import pandas as pd
import numpy as np
import os
import re
import requests
from bs4 import BeautifulSoup
import itertools

# import library
import scrape_ufc_stats_library as LIB
import importlib
importlib.reload(LIB)

# import configs
import yaml
config = yaml.safe_load(open('scrape_ufc_stats_config.yaml'))

# Parse Event Details
Includes:
<br>
Event
<br>
URL
<br>
Date
<br>
Location
<br>

In [146]:
# define url to parse
url = 'http://ufcstats.com/statistics/events/completed' # first page
# url = 'http://ufcstats.com/statistics/events/completed?page=all' # all pages

# get soup
soup = LIB.get_soup(url)

In [147]:
# parse event details
event_details_df = LIB.parse_event_details(soup)

# show event details
display(event_details_df)

# Parse Fight Details
Includes:
<br>
Event
<br>
Bout
<br>
URL

In [148]:
# parse one event for fight details

# define url to parse
url = 'http://ufcstats.com/event-details/509697e08673d2e5'
# get soup
soup = LIB.get_soup(url)

In [149]:
# parse fight links
fight_details_df = LIB.parse_fight_details(soup)

# show fight links
display(fight_details_df)

# Parse Fight Results and Stats

In [150]:
# define url to parse
# various types of fights
# url = 'http://ufcstats.com/fight-details/4b7ec02b39fc6f70' # one round finish
# url = 'http://ufcstats.com/fight-details/8b3b38167060b1d7' # three rounds decision
# url = 'http://ufcstats.com/fight-details/b22eab3aa1522f40' # three rounds finish
# url = 'http://ufcstats.com/fight-details/3109d1151f149aaf' # five rounds decision
# url = 'http://ufcstats.com/fight-details/d93c8c77e1091a16' # no stats
# url = 'http://ufcstats.com/fight-details/c63edd25d2201a46' # draw
url = 'http://ufcstats.com/fight-details/37cb7ce0f0b70640' # point deduction

# get soup
soup = LIB.get_soup(url)

### Parse Fight Results

Includes:
<br>
Event
<br>
Bout
<br>
Weightclass
<br>
Method
<br>
Round
<br>
Time
<br>
Time Format
<br>
Referee
<br>
Details
<br>

In [151]:
# parse fight results from soup
fight_results = LIB.parse_fight_results(soup)
# append fight url
fight_results.append('URL:'+url)

# show fight results
fight_results

In [152]:
# organise fight results
fight_results_df = LIB.organise_fight_results(fight_results, config['fight_results_column_names'])

# show fight results
display(fight_results_df)

### Parse Fight Stats
Includes:
Event
<br>
Bout
<br>
Round
<br>
Fighter
<br>
Kd
<br>
Sig.Str.
<br>
Sig.Str. %
<br>
Total Str.
<br>
Td
<br>
Td %
<br>
Sub.Att
<br>
Rev.
<br>
Ctrl
<br>
Head
<br>
Body
<br>
Leg
<br>
Distance
<br>
Clinch
<br>
Ground
<br>

In [153]:
# parse full fight stats for both fighters
fighter_a_stats, fighter_b_stats = LIB.parse_fight_stats(soup)

# show fighter stats
print(fighter_a_stats[:20])

In [154]:
# organise stats extracted from soup
fighter_a_stats_clean = LIB.organise_fight_stats(fighter_a_stats)
fighter_b_stats_clean = LIB.organise_fight_stats(fighter_b_stats)

# show organised stats
print(fighter_a_stats_clean[:2])

In [155]:
# convert list of fighter stats into a structured dataframe
fighter_a_stats_df = LIB.convert_fight_stats_to_df(fighter_a_stats_clean, config['totals_column_names'], config['significant_strikes_column_names'])
fighter_b_stats_df = LIB.convert_fight_stats_to_df(fighter_b_stats_clean, config['totals_column_names'], config['significant_strikes_column_names'])

# show stats df
display(fighter_a_stats_df)

In [156]:
# combine fighter stats into one
fight_stats = LIB.combine_fighter_stats_dfs(fighter_a_stats_df, fighter_b_stats_df, soup)

# show fight stats
display(fight_stats)

# Parse Fighter Details

In [56]:
# define url to parse
url = 'http://ufcstats.com/statistics/fighters?char=b&page=all' # 'a' last names
url = 'http://ufcstats.com/statistics/fighters?char=b&page=all' # 'b' last names

In [57]:
# get soup
soup = LIB.get_soup(url)

In [58]:
# parse fighter details
fighter_details_df = LIB.parse_fighter_details(soup, config['fighter_details_column_names'])

# show fighter details
display(fighter_details_df)

# Parse Fighter Tale Of The Tape

In [59]:
# define url to parse
url = 'http://ufcstats.com/fighter-details/93fe7332d16c6ad9' # '--' and '' present
url = 'http://ufcstats.com/fighter-details/b361180739bed4b0' # '--' present
url = 'http://ufcstats.com/fighter-details/1897b7b913736a7c' # no first name
url = 'http://ufcstats.com/fighter-details/d0f3959b4a9747e6' # all tott present

In [60]:
# get soup
soup = LIB.get_soup(url)

In [61]:
# parse fighter tale of the tape
fighter_tott = LIB.parse_fighter_tott(soup)

# show tale of the tape
fighter_tott

In [62]:
# create empty df to store fighters' tale of the tape
all_fighter_tott_df = pd.DataFrame(columns=config['fighter_tott_column_names'])

# organise fighter tale of the tape
fighter_tott_df = LIB.organise_fighter_tott(fighter_tott, config['fighter_tott_column_names'], url)

# show fighter tale of the tape
display(fighter_tott_df)